<a href="https://colab.research.google.com/github/ajayrawatsap/object_detection/blob/master/pothole_yolo3_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download code files and dataset

In [1]:
%%bash
git clone https://github.com/experiencor/keras-yolo3



Cloning into 'keras-yolo3'...


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r '/content/drive/My Drive/ML/pothole_detection/train_data.zip' '/content/'
!unzip 'train_data.zip' -d  '/content/'

In [0]:
# %%bash
# pip show keras
# pip show tensorflow

## Download Weights

In [5]:
import os
os.chdir('/content/keras-yolo3')
!wget https://s3-ap-southeast-1.amazonaws.com/deeplearning-mat/backend.h5
import tensorflow as tf
print(tf.__version__)

--2019-11-02 08:15:26--  https://s3-ap-southeast-1.amazonaws.com/deeplearning-mat/backend.h5
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.128.2
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.128.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248671664 (237M) [application/x-www-form-urlencoded]
Saving to: ‘backend.h5’

backend.h5          100%[===================>] 237.15M  13.6MB/s    in 19s     

2019-11-02 08:15:46 (12.3 MB/s) - ‘backend.h5’ saved [248671664/248671664]



1.15.0


## Change the config file

In [0]:
import json
with open("zoo/config_raccoon.json", "r") as jsonFile:
    data = json.load(jsonFile)
data['model']['labels'] = ['pothole']

#use script in next cell to generate anchors
# data['model']['anchors'] = [11,10, 13,18, 17,12, 20,18, 22,26, 26,47, 30,20, 38,33, 58,52]
data['model']['anchors'] = [ 11,10, 12,17, 17,12, 21,26, 24,18, 29,52, 36,31, 50,44, 63,63]

data['train']['train_image_folder'] = '/content/train_data/train_images/'
data['train']['train_annot_folder'] = '/content/train_data/train_annot/'



data['valid']['valid_image_folder'] = '/content/train_data/valid_images/'
data['valid']['valid_annot_folder'] = '/content/train_data/valid_annot/'
data['valid']['cache_name'] = 'road_valid.pkl'

data['train']['cache_name'] = 'pothole_train.pkl'
data['train']['nb_epochs'] = 50
data['train']['gpus'] = '0'
data['train']["batch_size"] = 8
data['train']['saved_weights_name'] = 'road.h5'
with open("config.json", "w") as jsonFile:
    json.dump(data, jsonFile)

In [0]:
# !python gen_anchors.py -c config.json

In [0]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

## Train


In [9]:
#! /usr/bin/env python



import os
os.chdir('/content/keras-yolo3')
import argparse

import numpy as np
import json
from voc import parse_voc_annotation
from yolo import create_yolov3_model, dummy_loss
from generator import BatchGenerator
from utils.utils import normalize, evaluate, makedirs
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from callbacks import CustomModelCheckpoint, CustomTensorBoard
from utils.multi_gpu_model import multi_gpu_model
import tensorflow as tf
import keras
from keras.models import load_model

tf.logging.set_verbosity(tf.logging.ERROR)


def create_training_instances(
    train_annot_folder,
    train_image_folder,
    train_cache,
    valid_annot_folder,
    valid_image_folder,
    valid_cache,
    labels,
):
    # parse annotations of the training set
    train_ints, train_labels = parse_voc_annotation(train_annot_folder, train_image_folder, train_cache, labels)

    # parse annotations of the validation set, if any, otherwise split the training set
    if os.path.exists(valid_annot_folder):
        valid_ints, valid_labels = parse_voc_annotation(valid_annot_folder, valid_image_folder, valid_cache, labels)
    else:
        print("valid_annot_folder not exists. Spliting the trainining set.")

        train_valid_split = int(0.8*len(train_ints))
        np.random.seed(0)
        np.random.shuffle(train_ints)
        np.random.seed()

        valid_ints = train_ints[train_valid_split:]
        train_ints = train_ints[:train_valid_split]

    # compare the seen labels with the given labels in config.json
    if len(labels) > 0:
        overlap_labels = set(labels).intersection(set(train_labels.keys()))

        print('Seen labels: \t'  + str(train_labels) + '\n')
        print('Given labels: \t' + str(labels))

        # return None, None, None if some given label is not in the dataset
        if len(overlap_labels) < len(labels):
            print('Some labels have no annotations! Please revise the list of labels in the config.json.')
            return None, None, None
    else:
        print('No labels are provided. Train on all seen labels.')
        print(train_labels)
        labels = train_labels.keys()
    print('test', train_ints)
    max_box_per_image = max([len(inst['object']) for inst in (train_ints + valid_ints)])

    return train_ints, valid_ints, sorted(labels), max_box_per_image

def create_callbacks(saved_weights_name, tensorboard_logs, model_to_save):
    makedirs(tensorboard_logs)
    
    early_stop = EarlyStopping(
        monitor     = 'loss', 
        min_delta   = 0.01, 
        patience    = 5, 
        mode        = 'min', 
        verbose     = 1
    )
    checkpoint = CustomModelCheckpoint(
        model_to_save   = model_to_save,
        filepath        = saved_weights_name,# + '{epoch:02d}.h5', 
        monitor         = 'loss', 
        verbose         = 1, 
        save_best_only  = True, 
        mode            = 'min', 
        period          = 1
    )
    reduce_on_plateau = ReduceLROnPlateau(
        monitor  = 'loss',
        factor   = 0.1,
        patience = 2,
        verbose  = 1,
        mode     = 'min',
        epsilon  = 0.01,
        cooldown = 0,
        min_lr   = 0
    )
    tensorboard = CustomTensorBoard(
        log_dir                = tensorboard_logs,
        write_graph            = True,
        write_images           = True,
    )    
    return [early_stop, checkpoint, reduce_on_plateau, tensorboard]

def create_model(
    nb_class, 
    anchors, 
    max_box_per_image, 
    max_grid, batch_size, 
    warmup_batches, 
    ignore_thresh, 
    multi_gpu, 
    saved_weights_name, 
    lr,
    grid_scales,
    obj_scale,
    noobj_scale,
    xywh_scale,
    class_scale  
):
    if multi_gpu > 1:
        with tf.device('/cpu:0'):
            template_model, infer_model = create_yolov3_model(
                nb_class            = nb_class, 
                anchors             = anchors, 
                max_box_per_image   = max_box_per_image, 
                max_grid            = max_grid, 
                batch_size          = batch_size//multi_gpu, 
                warmup_batches      = warmup_batches,
                ignore_thresh       = ignore_thresh,
                grid_scales         = grid_scales,
                obj_scale           = obj_scale,
                noobj_scale         = noobj_scale,
                xywh_scale          = xywh_scale,
                class_scale         = class_scale
            )
    else:
        template_model, infer_model = create_yolov3_model(
            nb_class            = nb_class, 
            anchors             = anchors, 
            max_box_per_image   = max_box_per_image, 
            max_grid            = max_grid, 
            batch_size          = batch_size, 
            warmup_batches      = warmup_batches,
            ignore_thresh       = ignore_thresh,
            grid_scales         = grid_scales,
            obj_scale           = obj_scale,
            noobj_scale         = noobj_scale,
            xywh_scale          = xywh_scale,
            class_scale         = class_scale
        )  

    # load the pretrained weight if exists, otherwise load the backend weight only
    if os.path.exists(saved_weights_name): 
        print("\nLoading pretrained weights.\n")
        template_model.load_weights(saved_weights_name)
    else:
        template_model.load_weights("backend.h5", by_name=True)       

    if multi_gpu > 1:
        train_model = multi_gpu_model(template_model, gpus=multi_gpu)
    else:
        train_model = template_model      

    optimizer = Adam(lr=lr, clipnorm=0.001)
    train_model.compile(loss=dummy_loss, optimizer=optimizer)             

    return train_model, infer_model

def _main_(config_path):
    config_path = config_path

    with open(config_path) as config_buffer:    
        config = json.loads(config_buffer.read())

    ###############################
    #   Parse the annotations 
    ###############################
    train_ints, valid_ints, labels, max_box_per_image = create_training_instances(
        config['train']['train_annot_folder'],
        config['train']['train_image_folder'],
        config['train']['cache_name'],
        config['valid']['valid_annot_folder'],
        config['valid']['valid_image_folder'],
        config['valid']['cache_name'],
        config['model']['labels']
    )
    print('\nTraining on: \t' + str(labels) + '\n')

    ###############################
    #   Create the generators 
    ###############################    
    train_generator = BatchGenerator(
        instances           = train_ints, 
        anchors             = config['model']['anchors'],   
        labels              = labels,        
        downsample          = 32, # ratio between network input's size and network output's size, 32 for YOLOv3
        max_box_per_image   = max_box_per_image,
        batch_size          = config['train']['batch_size'],
        min_net_size        = config['model']['min_input_size'],
        max_net_size        = config['model']['max_input_size'],   
        shuffle             = True, 
        jitter              = 0.3, 
        norm                = normalize
    )
    
    valid_generator = BatchGenerator(
        instances           = valid_ints, 
        anchors             = config['model']['anchors'],   
        labels              = labels,        
        downsample          = 32, # ratio between network input's size and network output's size, 32 for YOLOv3
        max_box_per_image   = max_box_per_image,
        batch_size          = config['train']['batch_size'],
        min_net_size        = config['model']['min_input_size'],
        max_net_size        = config['model']['max_input_size'],   
        shuffle             = True, 
        jitter              = 0.0, 
        norm                = normalize
    )

    ###############################
    #   Create the model 
    ###############################
    if os.path.exists(config['train']['saved_weights_name']): 
        config['train']['warmup_epochs'] = 0
    warmup_batches = config['train']['warmup_epochs'] * (config['train']['train_times']*len(train_generator))   

    os.environ['CUDA_VISIBLE_DEVICES'] = config['train']['gpus']
    multi_gpu = len(config['train']['gpus'].split(','))

    train_model, infer_model = create_model(
        nb_class            = len(labels), 
        anchors             = config['model']['anchors'], 
        max_box_per_image   = max_box_per_image, 
        max_grid            = [config['model']['max_input_size'], config['model']['max_input_size']], 
        batch_size          = config['train']['batch_size'], 
        warmup_batches      = warmup_batches,
        ignore_thresh       = config['train']['ignore_thresh'],
        multi_gpu           = multi_gpu,
        saved_weights_name  = config['train']['saved_weights_name'],
        lr                  = config['train']['learning_rate'],
        grid_scales         = config['train']['grid_scales'],
        obj_scale           = config['train']['obj_scale'],
        noobj_scale         = config['train']['noobj_scale'],
        xywh_scale          = config['train']['xywh_scale'],
        class_scale         = config['train']['class_scale'],
    )

    ###############################
    #   Kick off the training
    ###############################
    callbacks = create_callbacks(config['train']['saved_weights_name'], config['train']['tensorboard_dir'], infer_model)
    print('Training started')
    train_model.fit_generator(
        generator        = train_generator, 
        steps_per_epoch  = len(train_generator) * config['train']['train_times'], 
        epochs           = config['train']['nb_epochs'] + config['train']['warmup_epochs'], 
        verbose          = 1 ,#2 if config['train']['debug'] else 1,
        callbacks        = callbacks, 
        validation_data = valid_generator,
        validation_steps = len(valid_generator),
        workers          = 4,
        max_queue_size   = 8
    )
     
    # make a GPU version of infer_model for evaluation
    if multi_gpu > 1:
        infer_model = load_model(config['train']['saved_weights_name'])

    ###############################
    #   Run the evaluation
    ###############################   
    # compute mAP for all the classes
    average_precisions = evaluate(infer_model, valid_generator)

    # print the score
    for label, average_precision in average_precisions.items():
        print(labels[label] + ': {:.4f}'.format(average_precision))
    print('mAP: {:.4f}'.format(sum(average_precisions.values()) / len(average_precisions)))           

if __name__ == '__main__':
#    argparser = argparse.ArgumentParser(description='train and evaluate YOLO_v3 model on any dataset')
#    argparser.add_argument('-c', '--conf', help='path to configuration file')   

#    args = argparser.parse_args()
    config_path = 'config.json'
    _main_(config_path)


Using TensorFlow backend.


Seen labels: 	{'pothole': 450}

Given labels: 	['pothole']
test [{'object': [{'name': 'pothole', 'xmin': 843, 'ymin': 501, 'xmax': 884, 'ymax': 523}, {'name': 'pothole', 'xmin': 447, 'ymin': 486, 'xmax': 491, 'ymax': 509}], 'filename': '/content/train_data/train_images/scene00421.jpg', 'width': 1280, 'height': 720}, {'object': [{'name': 'pothole', 'xmin': 432, 'ymin': 474, 'xmax': 539, 'ymax': 533}], 'filename': '/content/train_data/train_images/scene00436.jpg', 'width': 1280, 'height': 720}, {'object': [{'name': 'pothole', 'xmin': 599, 'ymin': 471, 'xmax': 730, 'ymax': 522}, {'name': 'pothole', 'xmin': 623, 'ymin': 413, 'xmax': 689, 'ymax': 443}], 'filename': '/content/train_data/train_images/scene00601.jpg', 'width': 1280, 'height': 720}, {'object': [{'name': 'pothole', 'xmin': 627, 'ymin': 498, 'xmax': 706, 'ymax': 538}, {'name': 'pothole', 'xmin': 665, 'ymin': 433, 'xmax': 691, 'ymax': 460}, {'name': 'pothole', 'xmin': 630, 'ymin': 404, 'xmax': 695, 'ymax': 428}], 'filename': '/con

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1335: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Training started
Epoch 1/53
resizing:  384 384
83/84 [============================>.] - ETA: 1s - loss: 592.0197 - yolo_layer_1_loss: 74.5771 - yolo_layer_2_loss: 159.4469 - yolo_layer_3_loss: 357.9957resizing:  352 352
resizing:  384 384
resizing:  320 320
84/84 [==============================] - 146s 2s/step - loss: 587.0896 - yolo_layer_1_loss: 73.9684 - yolo_layer_2_loss: 158.0946 - yolo_layer_3_loss: 355.0266 - val_loss: 1141.0342 - val_yolo_layer_1_loss: 116.3157 - val_yolo_layer_2_loss: 277.8780 - val_yolo_layer_3_loss: 746.8405

Epoch 00001: loss improved from inf to 587.08955, saving model to road.h5
Epoch 2/53
resizing:  416 416
83/84 [============================>.] - ETA: 1s - loss: 126.4057 - yolo_layer_1_loss: 18.0549 - yolo_layer_2_loss: 31.3462 - yolo_layer_3_loss: 77.0046resizing:  384 384
resizing:  320 320
84/84 [==============================] - 111s 1s/step - loss: 125.8879 - yolo_layer_1_loss: 17.9948 - yolo_layer_2_loss: 31.2378 - yolo_layer_3_loss: 76.6554 - val

## Predict Image



In [10]:
import os
from keras.models import load_model
# config_path  = 'config.json'
# with open(config_path) as config_buffer:    
#     config = json.load(config_buffer)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# infer_model = load_model(config['train']['saved_weights_name'])
infer_model = load_model('/content/drive/My Drive/ML/pothole_detection/road.h5')
os.chdir('keras-yolo3')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


FileNotFoundError: ignored

In [0]:

%%time


import os
import argparse
import json
import cv2
from utils.utils import get_yolo_boxes, makedirs
from utils.bbox import draw_boxes
import matplotlib.pyplot as plt

from tqdm import tqdm
import numpy as np

import time
from urllib.request import urlopen

def url_to_image(url, readFlag=cv2.IMREAD_COLOR):
    # download the image, convert it to a NumPy array, and then read
    # it into OpenCV format
    resp = urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, readFlag)

    # return the image
    return image


input_path   = '../drive/My Drive/ML/pothole_detection/test/'
# input_path =   'https://i.dailymail.co.uk/i/pix/2014/03/03/article-2572043-1BFE7DCF00000578-273_634x753.jpg'
output_path  =  '../drive/My Drive/ML/output/'



makedirs(output_path)

 
net_h, net_w = 416, 416 # a multiple of 32, the smaller the faster
obj_thresh, nms_thresh = 0.5, 0.45

image_paths = []

if os.path.isdir(input_path): 
    for inp_file in os.listdir(input_path):
        image_paths += [input_path + inp_file]
else:
    image_paths += [input_path]

image_paths = [inp_file for inp_file in image_paths if (inp_file[-4:] in ['.jpg', '.png', 'JPEG'])]

print('test', image_paths)
# the main loop
for image_path in image_paths:
    start = time.time()
    image = cv2.imread(image_path)
    # image = url_to_image(image_path)
    
    # predict the bounding boxes
    boxes = get_yolo_boxes(infer_model, [image], net_h, net_w, config['model']['anchors'], obj_thresh, nms_thresh)[0]

    # draw bounding boxes on the image using labels
    draw_boxes(image, boxes, config['model']['labels'], obj_thresh) 
    print('detection time', time.time() - start)
    # write the image with bounding boxes to file
    cv2.imwrite(output_path + image_path.split('/')[-1], np.uint8(image))   
       
    plt.figure(figsize=(15, 15))
    plt.axis('off')
    plt.imshow(image)
    plt.show()
    
    


## Predict Video
